In [133]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [195]:
# Подключаем к наше программе модуль с предыдущего шага. Этот модуль будет повторяться всегда. 
# Мы импортируем модуль psycopg2 и называем его pg_driver для удобства
import psycopg2 as pg_driver
from psycopg2.extras import NamedTupleCursor


# На данном шаге нам нужно попробовать подключить к нашей базе данных. При установке PostgreSQL вам предлагалось задать
# пароль от базы данных по умолчанию, собственно ниже вам следует изменить только password='postgres', но не удаляйте кавычки, это важно
db = pg_driver.connect(
    database="testdb_2", 
    user='testuser_2',
    password='testuser_2', 
    host='localhost', 
    port='5432'
)

# Далее нам нужен курсор, который позволит нам непосредственно отправлять запросы на выполнения в БД. Эта строка всегда одинаковая
cur = db.cursor()
# Делаем запрос к таблице по умолчанию pg_database
cur.execute("select * from pg_database")

# Чтобы извлечь результат выполнения запроса к таблице по умолчанию, необходимо воспользоваться методом .fetchall()
rs = cur.fetchall()


# enumerate позволяет красиво пронумеровать строки
for row, values in enumerate(rs):
    print(row, values)
              

0 (13754, 'postgres', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
1 (16394, 'myfirstdb', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
2 (1, 'template1', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', True, True, -1, 13753, '727', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
3 (13753, 'template0', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', True, False, -1, 13753, '727', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
4 (16406, 'testdb', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
5 (16407, 'testdb_1', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
6 (16408, 'testdb_2', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
7 (16395, 'testdba', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', Fal

In [196]:
import psycopg2
from psycopg2 import Error

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                                database="testdb_2", 
                                user='testuser_2',
                                password='testuser_2', 
                                host='localhost', 
                                port='5432'
                     )
        connection.autocommit = True
    
        cursor = connection.cursor(cursor_factory = NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [246]:
import psycopg2 as pg_driver

db = psycopg2.connect(database="testdb_2",
                      user='testuser_2',
                      password='testuser_2', 
                      host='localhost', 
                      port='5432'
                )


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         surname TEXT  NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         surname TEXT  NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [249]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="testdb_2", 
                        user='testuser_2',
                        password='testuser_2', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=20)]
[Record(count=0)]


In [248]:
query = """ insert into users (user_id, created, name, surname)
            values
                 (1, to_timestamp('20-01-2022 14:12:18', 'dd-mm-yyyy hh24:mi:ss'), 'Sean', 'Penn'),
                 (2, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), 'Tom', 'Cruise'),
                 (3, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), 'Paul', 'Newman'),
                 (4, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), 'Robin', 'Williams'),
                 (5, to_timestamp('16-06-2020 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), 'Robert', 'De Niro' ),
                 (6, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), 'Christian', 'Bale'),
                 (7, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), 'Dustin', 'Hoffman'),
                 (8, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), 'Denzel', 'Washington'),
                 (9, to_timestamp('16-06-2020 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), 'Tom', 'Hanks'),
                 (10, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), 'Daniel', 'Day-Lewis'),
                 (11, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), 'Brad', 'Pitt'),
                 (12, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), 'Harrison', 'Ford'),
                 (13, to_timestamp('20-01-2022 14:12:18', 'dd-mm-yyyy hh24:mi:ss'), 'Rassel', 'Crowe'),
                 (14, to_timestamp('26-05-2021 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), 'Joaquin', 'Phoenix'),
                 (15, to_timestamp('17-07-2016 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), 'Clint', 'Eastwood'),
                 (16, to_timestamp('30-12-2015 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), 'Robin', 'Williams'),
                 (17, to_timestamp('16-06-2007 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), 'Morgan', 'Freeman' ),
                 (18, to_timestamp('26-05-2002 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), 'Al', 'Pachino'),
                 (19, to_timestamp('27-10-2012 05:17:28', 'dd-mm-yyyy hh24:mi:ss'), 'Leonardo', 'Dicaprio'),
                 (20, to_timestamp('23-11-2011 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), 'Zendaya', '')
                 
        """
execute_query(query)
  

In [250]:
query = """ insert into likes (user_id, created, user_id_give)
            values
                 (1, to_timestamp('20-01-2022 14:12:18', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (2, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (3, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (4, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (5, to_timestamp('16-06-2020 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), '20' ),
                 (6, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (7, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (8, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (9, to_timestamp('16-06-2020 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (10, to_timestamp('26-05-2018 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (11, to_timestamp('17-07-2020 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (12, to_timestamp('30-12-2010 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (13, to_timestamp('20-01-2022 14:12:18', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (14, to_timestamp('26-05-2021 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (15, to_timestamp('17-07-2016 09:47:32', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (16, to_timestamp('30-12-2015 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (17, to_timestamp('16-06-2007 23:13:29', 'dd-mm-yyyy hh24:mi:ss'), '20' ),
                 (18, to_timestamp('26-05-2002 04:51:44', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (19, to_timestamp('27-10-2012 05:17:28', 'dd-mm-yyyy hh24:mi:ss'), '20'),
                 (20, to_timestamp('23-11-2011 11:06:01', 'dd-mm-yyyy hh24:mi:ss'), '4')
                 
        """
execute_query(query)

In [228]:
all_rows = execute_query("select * from likes", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, created=datetime.datetime(2022, 1, 20, 14, 12, 18), user_id_give=20)
1 Record(user_id=2, created=datetime.datetime(2018, 5, 26, 4, 51, 44), user_id_give=20)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 17, 9, 47, 32), user_id_give=20)
3 Record(user_id=4, created=datetime.datetime(2010, 12, 30, 11, 6, 1), user_id_give=20)
4 Record(user_id=5, created=datetime.datetime(2020, 6, 16, 23, 13, 29), user_id_give=20)
5 Record(user_id=6, created=datetime.datetime(2018, 5, 26, 4, 51, 44), user_id_give=20)
6 Record(user_id=7, created=datetime.datetime(2020, 7, 17, 9, 47, 32), user_id_give=20)
7 Record(user_id=8, created=datetime.datetime(2010, 12, 30, 11, 6, 1), user_id_give=20)
8 Record(user_id=9, created=datetime.datetime(2020, 6, 16, 23, 13, 29), user_id_give=20)
9 Record(user_id=10, created=datetime.datetime(2018, 5, 26, 4, 51, 44), user_id_give=20)
10 Record(user_id=11, created=datetime.datetime(2020, 7, 17, 9, 47, 32), user_id_give=20)
11 Record(user_id=12, crea

In [231]:
all_rows = execute_query("select user_id, count(*) from likes where user_id=1 group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, count=1)


In [232]:
query = """select
                l.user_id as from_id,
                u.name as from_name,
                u.surname as from_surname,
                l.user_id_give as to_id,
                us.name as to_name
           from likes l
           left join users u
           on l.user_id=u.user_id
           left join users us
           on l.user_id_give=us.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(from_id=20, from_name='Zendaya', from_surname='', to_id=4, to_name='Robin')
1 Record(from_id=2, from_name='Tom', from_surname='Cruise', to_id=20, to_name='Zendaya')
2 Record(from_id=3, from_name='Paul', from_surname='Newman', to_id=20, to_name='Zendaya')
3 Record(from_id=4, from_name='Robin', from_surname='Williams', to_id=20, to_name='Zendaya')
4 Record(from_id=5, from_name='Robert', from_surname='De Niro', to_id=20, to_name='Zendaya')
5 Record(from_id=6, from_name='Christian', from_surname='Bale', to_id=20, to_name='Zendaya')
6 Record(from_id=7, from_name='Dustin', from_surname='Hoffman', to_id=20, to_name='Zendaya')
7 Record(from_id=8, from_name='Denzel', from_surname='Washington', to_id=20, to_name='Zendaya')
8 Record(from_id=9, from_name='Tom', from_surname='Hanks', to_id=20, to_name='Zendaya')
9 Record(from_id=10, from_name='Daniel', from_surname='Day-Lewis', to_id=20, to_name='Zendaya')
10 Record(from_id=1, from_name='Sean', from_surname='Penn', to_id=20, to_name='Zenda

In [240]:
query = """select
                l.user_id as from_id,
                u.name as sender_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by l.user_id, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(from_id=8, sender_name='Denzel', like_count=1)
1 Record(from_id=10, sender_name='Daniel', like_count=1)
2 Record(from_id=9, sender_name='Tom', like_count=1)
3 Record(from_id=2, sender_name='Tom', like_count=2)
4 Record(from_id=11, sender_name='Brad', like_count=1)
5 Record(from_id=12, sender_name='Harrison', like_count=1)
6 Record(from_id=7, sender_name='Dustin', like_count=1)
7 Record(from_id=20, sender_name='Zendaya', like_count=1)
8 Record(from_id=17, sender_name='Morgan', like_count=1)
9 Record(from_id=19, sender_name='Leonardo', like_count=1)
10 Record(from_id=15, sender_name='Clint', like_count=1)
11 Record(from_id=3, sender_name='Paul', like_count=1)
12 Record(from_id=16, sender_name='Robin', like_count=1)
13 Record(from_id=13, sender_name='Rassel', like_count=1)
14 Record(from_id=6, sender_name='Christian', like_count=1)
15 Record(from_id=4, sender_name='Robin', like_count=1)
16 Record(from_id=18, sender_name='Al', like_count=1)
17 Record(from_id=1, sender_name='Sean',

In [310]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 

                 
            
        """

execute_query(query)

In [267]:
query = """select user_id, count (user_id_give) from likes where user_id_give = '20' group by user_id
           
        """
# Кто сколько лайков поставил Зендае

all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, count=3)
1 Record(user_id=2, count=2)
2 Record(user_id=3, count=1)
3 Record(user_id=4, count=1)
4 Record(user_id=5, count=1)
5 Record(user_id=6, count=1)
6 Record(user_id=7, count=1)
7 Record(user_id=8, count=1)
8 Record(user_id=9, count=1)
9 Record(user_id=10, count=1)
10 Record(user_id=11, count=1)
11 Record(user_id=12, count=1)
12 Record(user_id=13, count=1)
13 Record(user_id=14, count=1)
14 Record(user_id=15, count=1)
15 Record(user_id=16, count=1)
16 Record(user_id=17, count=1)
17 Record(user_id=18, count=1)
18 Record(user_id=19, count=1)


In [313]:
query = """select
                u.user_id as sender_id,
                u.name as sender_name,
                u.surname as sender_surname,
                l.user_id_give as recipient_id,
                
                count(*) as amount_of_sent_likes
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           
           group by l.user_id_give, u.name, u.surname, u.user_id order by u.user_id
           
           
        """
# Кому и сколько пользователь оптарвлял лайки.

all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(sender_id=1, sender_name='Sean', sender_surname='Penn', recipient_id=1, amount_of_sent_likes=1)
1 Record(sender_id=1, sender_name='Sean', sender_surname='Penn', recipient_id=20, amount_of_sent_likes=3)
2 Record(sender_id=2, sender_name='Tom', sender_surname='Cruise', recipient_id=20, amount_of_sent_likes=2)
3 Record(sender_id=3, sender_name='Paul', sender_surname='Newman', recipient_id=20, amount_of_sent_likes=1)
4 Record(sender_id=4, sender_name='Robin', sender_surname='Williams', recipient_id=20, amount_of_sent_likes=1)
5 Record(sender_id=5, sender_name='Robert', sender_surname='De Niro', recipient_id=20, amount_of_sent_likes=1)
6 Record(sender_id=6, sender_name='Christian', sender_surname='Bale', recipient_id=20, amount_of_sent_likes=1)
7 Record(sender_id=7, sender_name='Dustin', sender_surname='Hoffman', recipient_id=20, amount_of_sent_likes=1)
8 Record(sender_id=8, sender_name='Denzel', sender_surname='Washington', recipient_id=20, amount_of_sent_likes=1)
9 Record(sender_

In [1]:
print("('P00" + str(i) + "', " + "'" + str(create_data()) + "', '" + column_1[random.randint(0, len(column_1))-1] + "', '" + column_2[random.randint(0, len(column_2))-1] + "'), ")


NameError: name 'i' is not defined